In [1]:
!python --version

Python 3.13.5


In [6]:
# === Импорты и настройки ===

import os
import pandas as pd
from datetime import datetime as dt
from statsmodels.tsa.api import VAR
from statsmodels.tsa.x13 import x13_arima_analysis  # нужен установленный X-13

# Если ноутбук в одной папке с файлами, оставь точку:
fd_ru = "."  # текущая папка
# иначе пропиши полный путь, например:
# fd_ru = "/Users/uliana/Documents/Inflation"

X13PATH = "./x13as/x13as_html"

In [7]:
def clean_numeric(df: pd.DataFrame) -> pd.DataFrame:
    for c in df.columns:
        s = df[c]
        if pd.api.types.is_numeric_dtype(s):
            continue
        df[c] = pd.to_numeric(s.astype(str).str.strip().str.replace(',', '.'), errors='coerce')
    return df

In [8]:
# === Список категорий ===

map_path = os.path.join(fd_ru, "PriceConsumer.xlsx")
df_pi2rs = (
    pd.read_excel(map_path, sheet_name="MAP_PI_RS")
      .query("TYPE == 'GOODS'")
      .dropna(subset=['Pi'])
)

CATEGORIES = df_pi2rs['Pi'].tolist()
CATEGORIES

['Meat',
 'Fish',
 'Cheese',
 'Eggs',
 'Sugar',
 'Pastries',
 'Bread',
 'Groats',
 'Fruit']

In [22]:
# === ВЕСА ===

w_path = os.path.join(fd_ru, "PriceConsumer.xlsx")
df_w = (
    pd.read_excel(w_path, sheet_name="W_BB", index_col='ds')
      .resample('M', closed='right').ffill()   # растянуть годовые точки на месяцы
      [CATEGORIES]
      .pipe(clean_numeric)
      .pipe(lambda df: df.div(df.sum(axis=1), axis=0))   # сумма по строке = 1
      .shift(1, freq='M')                                # лаг на 1 месяц
)
df_w.head()
df_w.tail()

/var/folders/8_/_hgc9svn1339rgsv79hwhw_80000gn/T/ipykernel_26203/3137624535.py:6: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample('M', closed='right').ffill()   # растянуть годовые точки на месяцы
/var/folders/8_/_hgc9svn1339rgsv79hwhw_80000gn/T/ipykernel_26203/3137624535.py:10: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .shift(1, freq='M')                                # лаг на 1 месяц


,Meat,Fish,Cheese,Eggs,Sugar,Pastries,Bread,Groats,Fruit
ds,,,,,,,,,
2024-10-31,0.387064,0.093779,0.056063,0.028074,0.015146,0.110718,0.064937,0.048212,0.196006
2024-11-30,0.387064,0.093779,0.056063,0.028074,0.015146,0.110718,0.064937,0.048212,0.196006
2024-12-31,0.387064,0.093779,0.056063,0.028074,0.015146,0.110718,0.064937,0.048212,0.196006
2025-01-31,0.387064,0.093779,0.056063,0.028074,0.015146,0.110718,0.064937,0.048212,0.196006
2025-02-28,0.370859,0.096561,0.057466,0.024929,0.013452,0.109971,0.065033,0.046326,0.215403


In [10]:
# === ИНФЛЯЦИЯ m/m SA (%) ===

pi_path = os.path.join(fd_ru, "PriceConsumerSeasonalAdjustmentBankOfRussia.xlsx")
pi_mm_sa = (
    pd.read_excel(pi_path, sheet_name="MM_SA_BR", index_col='ds')
      .resample('M').last()
      [CATEGORIES]
      .pipe(clean_numeric)
)
pi_mm_sa.head()

/var/folders/8_/_hgc9svn1339rgsv79hwhw_80000gn/T/ipykernel_26203/3552642571.py:6: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample('M').last()


,Meat,Fish,Cheese,Eggs,Sugar,Pastries,Bread,Groats,Fruit
ds,,,,,,,,,
2010-02-28,0.16,0.739185,3.244207,4.637891,1.792084,0.57,0.23,0.93,1.682562
2010-03-31,0.04,0.491735,3.156245,8.548764,-2.287639,0.41,0.25,0.43,1.623757
2010-04-30,0.10,-0.212407,2.289970,-2.107321,-4.216032,0.41,0.14,0.13,0.182556
2010-05-31,0.32,-0.363629,1.177825,-5.120316,-4.186651,0.31,0.06,-0.58,4.253894
2010-06-30,0.49,-0.384384,0.127206,-1.798777,-2.612087,0.25,0.09,0.93,2.682147


In [15]:
# === ОБЪЁМЫ: читаем SA уровни (осезонено руками) и считаем m/m, % ===

q_path = os.path.join(fd_ru, "TradeRetail.xlsx")

df_sa_levels = (
    pd.read_excel(q_path, sheet_name="RS_CAT_MM_RS", index_col='ds')
      [CATEGORIES]
      .pipe(clean_numeric)
)

# m/m, %
df_rs_mm_sa = df_sa_levels.pct_change().mul(100.0)

df_rs_mm_sa.head()

,Meat,Fish,Cheese,Eggs,Sugar,Pastries,Bread,Groats,Fruit
ds,,,,,,,,,
2010-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-03-31,1.569703,-0.495483,-0.863070,1.845171,1.541873,-2.416590,-1.387446,0.814733,-2.479258
2010-04-30,1.547673,2.476211,2.055201,-6.516132,6.815229,5.412610,1.223119,3.630592,3.035856
2010-05-31,0.239295,4.738032,-0.092787,7.632050,4.142267,0.366821,2.256132,2.579321,0.177445
2010-06-30,0.125044,1.251226,0.651349,2.690286,-7.053540,2.117122,1.052051,-1.034107,-1.106312


In [16]:
# === Окно дат и заготовки ===

dt_start = dt(2010, 3, 31)
dt_end   = dt(2025, 7, 31)
idx = pd.date_range(start=dt_start, end=dt_end, freq='M')

df_res_q = pd.DataFrame(index=idx, columns=CATEGORIES, dtype='float64')
df_res_p = pd.DataFrame(index=idx, columns=CATEGORIES, dtype='float64')
df_decomp = pd.DataFrame(index=idx, columns=['Supply','Demand'], dtype='float64')

len(idx), idx.min(), idx.max()

/var/folders/8_/_hgc9svn1339rgsv79hwhw_80000gn/T/ipykernel_26203/1636403174.py:5: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  idx = pd.date_range(start=dt_start, end=dt_end, freq='M')


(185, Timestamp('2010-03-31 00:00:00'), Timestamp('2025-07-31 00:00:00'))

In [17]:
for prod in CATEGORIES:
    print(prod)
    df_pq = pd.concat([
        pi_mm_sa.loc[dt_start:dt_end, [prod]].rename(columns={prod: 'P'}),
        df_rs_mm_sa.loc[dt_start:dt_end, [prod]].rename(columns={prod: 'Q'})
    ], axis=1)

    model = VAR(df_pq)
    lag = 4
    results = model.fit(lag)

    # запись остатков — как в исходнике
    df_res_q[prod] = results.resid.Q
    df_res_p[prod] = results.resid.P

Meat
Fish
Cheese
Eggs
Sugar
Pastries
Bread
Groats
Fruit


In [18]:
# Check - показать первые/последние строки для одной категории

cat0 = CATEGORIES[0]
display(df_res_p[[cat0]].join(df_res_q[[cat0]], lsuffix='_P', rsuffix='_Q').head(6))
display(df_res_p[[cat0]].join(df_res_q[[cat0]], lsuffix='_P', rsuffix='_Q').tail(6))

,Meat_P,Meat_Q
2010-03-31,NaN,NaN
2010-04-30,NaN,NaN
2010-05-31,NaN,NaN
2010-06-30,NaN,NaN
2010-07-31,0.166148,-0.933202
2010-08-31,-0.033146,0.559751


,Meat_P,Meat_Q
2025-02-28,-0.090677,0.651701
2025-03-31,-0.084270,-1.726975
2025-04-30,-0.144517,-0.367197
2025-05-31,0.732255,-0.217030
2025-06-30,-0.608603,-1.637952
2025-07-31,0.068450,-0.368224


In [23]:
df_shocks = pd.concat({"P": df_res_p, "Q": df_res_q}, axis=1)
df_shocks.to_excel("all_shocks.xlsx")

In [20]:
# маска знаков: >0 одинаковые (спрос), <0 разные (предложение)
df_mask = df_res_q * df_res_p

df_dem = pi_mm_sa.mask(df_mask < 0, 0) 
df_sup = pi_mm_sa.mask(df_mask > 0, 0)

df_decomp['Demand'] = df_w.mul(df_dem).sum(axis=1)
df_decomp['Supply'] = df_w.mul(df_sup).sum(axis=1)